### Generate datapoints

In [1]:
import numpy as np
import scipy as sp

def _init(m_dash, n_dash):
    # Step 1 or 5
    A = np.random.rand(m_dash, n_dash)
    Q, _ = sp.linalg.qr(A)
    return Q

m, n = 10, 10

silhouette = np.array([
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
])

# 5 silhouettes per datapoint
# Same for all, for the sake of simplicity
gait_sequence = np.array([silhouette for _ in range(5)])

datapoints = []
for i in range(10):
    # Subject ID assigned to each
    subject_id = f'00{i%3 + 1}'
    # Same for all, for the sake of simplicity
    conditions = 'concrete,backpack,sneakers,1_week'
    # Create the datapoint as a structured array
    datapoint = np.array([(subject_id, conditions, gait_sequence)], 
                         dtype=[('subject_id', 'U10'), ('conditions', 'U50'), ('gait_sequence', 'O')])
    # Append the datapoint to our list
    datapoints.append(datapoint)

dataset = np.concatenate(datapoints)

# print(dataset)

### Initialization

In [2]:
Tmax_c = 3
Tmax_d = 3
error = 3
m_dash_c = 5
n_dash_c = 5
m_dash_d = 3
n_dash_d = 3

import humangait
model = humangait.CSA_DATER()
model.train(dataset)

(array([[-0.0850991 +0.08155488j,  0.08490312+0.04616224j,
         -0.14886125-0.00209513j],
        [ 0.09658771+0.05175975j, -0.01018189-0.10904837j,
          0.20149154+0.12563764j],
        [-0.14409981+0.01246592j,  0.16028601+0.14224731j,
         -0.19499782-0.25949985j],
        [ 0.15080917-0.29229093j, -0.31834079-0.08164537j,
          0.25133914+0.06561643j],
        [-0.21641082+0.69907642j,  0.60017246+0.01063126j,
         -0.41418903+0.14238132j],
        [ 0.15080917-0.29229093j, -0.31834079-0.08164537j,
          0.25133914+0.06561643j],
        [-0.14409981+0.01246592j,  0.16028601+0.14224731j,
         -0.19499782-0.25949985j],
        [ 0.09658771+0.05175975j, -0.01018189-0.10904837j,
          0.20149154+0.12563764j],
        [-0.0850991 +0.08155488j,  0.08490312+0.04616224j,
         -0.14886125-0.00209513j],
        [ 0.26273407-0.29228685j, -0.49298856-0.21472633j,
          0.39424537+0.3872567j ]]),
 array([[-0.23751182+0.25500261j,  0.01341595+0.40661841j,

### CSA

In [ ]:
shape = dataset[0]["gait_sequence"][0].shape

m, n = shape
U = _init(m, m_dash_c)
for t in range(Tmax_c):
    # Step 2(a)
    F = []
    for datapoint in dataset:
        for silhouette in datapoint["gait_sequence"]:
            F += [U @ U.T @ silhouette]
    F = np.concatenate(F, axis=1)
    eigval, eigvec = sp.linalg.eig(F @ F.T)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    V_new = eigvec[:, :n_dash_c]
    # Step 2(b)
    G = []
    for datapoint in dataset:
        for silhouette in datapoint["gait_sequence"]:
            G += [silhouette @ V_new @ V_new.T]
    G = np.concatenate(G, axis=1)
    eigval, eigvec = sp.linalg.eig(G @ G.T)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    U_new = eigvec[:, :m_dash_c]
    # Step 2(c)
    if t > 1:
        U_error = np.linalg.norm(U_new - U, "fro")
        V_error = np.linalg.norm(V_new - V, "fro")
        if U_error < 10 * error and V_error < 10 * error:
            U, V = U_new, V_new
            break
    U, V = U_new, V_new

U_c, V_c = U, V
print(U, V)

### Use CSA Results

In [ ]:
# Step 4
projected_dataset = []
for datapoint in dataset:
    new_datapoint = datapoint.copy()
    projected_gait_sequence = []
    for silhouette in datapoint["gait_sequence"]:
        projected_gait_sequence.append(U.T @ silhouette @ V)
    new_datapoint["gait_sequence"] = projected_gait_sequence
    projected_dataset.append(new_datapoint)

dataset = projected_dataset
print(projected_dataset)

### DATER

In [ ]:
shape = dataset[0]["gait_sequence"][0].shape

m, n = shape
U = _init(m, m_dash_d)
# Group by class and calculate overall mean
classes = dict()
Xavg = []
for datapoint in dataset:
    label = datapoint["subject_id"]
    if label not in classes.keys():
        classes[label] = dict()
        classes[label]["samples"] = list()
    classes[label]["samples"] += datapoint["gait_sequence"]
    Xavg += datapoint["gait_sequence"]
Xavg = np.mean(Xavg, axis=0)
# Calculate mean of samples by class
for label, class_info in classes.items():
    class_info["Xavg_c"] = np.mean(class_info["samples"], axis=0)
for t in range(Tmax_d):
    # Step 6(a)
    Xuavg = U.T @ Xavg
    Su_b, Su_w = [None] * 2
    for label, class_info in classes.items():
        Xuavg_c = U.T @ class_info["Xavg_c"]
        n_c = len(class_info["samples"])
        delta = n_c * ((Xuavg_c - Xuavg).T @ (Xuavg_c - Xuavg))
        Su_b = delta if Su_b is None else Su_b + delta
        for silhouette in class_info["samples"]:
            Xu_i = U.T @ silhouette
            delta = (Xu_i - Xuavg_c).T @ (Xu_i - Xuavg_c)
            Su_w = delta if Su_w is None else Su_w + delta
    eigval, eigvec = sp.linalg.eig(Su_b, Su_w)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    V_new = eigvec[:, :n_dash_d]
    # Step 6(b)
    Xvavg = Xavg @ V_new
    Sv_b, Sv_w = [None] * 2
    for label, class_info in classes.items():
        Xvavg_c = class_info["Xavg_c"] @ V_new
        delta = n_c * ((Xvavg_c - Xvavg) @ (Xvavg_c - Xvavg).T)
        Sv_b = delta if Sv_b is None else Sv_b + delta
        for silhouette in class_info["samples"]:
            Xv_i = silhouette @ V_new
            delta =  (Xv_i - Xvavg_c) @ (Xv_i - Xvavg_c).T
            Sv_w = delta if Sv_w is None else Sv_w + delta
    eigval, eigvec = sp.linalg.eig(Sv_b, Sv_w)
    eigvec = eigvec[:, np.argsort(eigval)[::-1]]
    U_new = eigvec[:, :m_dash_d]
    # Step 6(c)
    if t > 1:
        U_error = np.linalg.norm(U_new - U, "fro")
        V_error = np.linalg.norm(V_new - V, "fro")
        if U_error < m * error and V_error < n * error:
            U, V = U_new, V_new
            break
    U, V = U_new, V_new

U_d, V_d = U, V
print(U, V)

In [ ]:
print(U_c @ U_d, V_c @ V_d)